In [3]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np

def run_inference_and_print_top100(model_path, X_test, y_test, test_df, max_length=512, batch_size=8, print_results=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    encoded = tokenizer(X_test, padding=True, truncation=True, max_length=max_length, return_tensors='pt')

    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    model.eval()
    all_preds = []

    with torch.no_grad():
        for start_idx in range(0, len(X_test), batch_size):
            end_idx = min(start_idx + batch_size, len(X_test))
            batch_input_ids = input_ids[start_idx:end_idx]
            batch_attention_mask = attention_mask[start_idx:end_idx]

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            batch_preds = torch.sigmoid(outputs.logits.squeeze()).cpu().numpy()  # Sigmoid for regression output in [0,1]
            all_preds.extend(batch_preds)

    all_preds = np.array(all_preds)

    names = test_df['name'].values

    # Top 100 predictions by predicted value
    top_pred_indices = np.argsort(all_preds)[-100:][::-1]
    top_100_pred_preds = all_preds[top_pred_indices]
    top_100_pred_actuals = np.array(y_test)[top_pred_indices]
    top_100_pred_names = names[top_pred_indices]

    # Top 100 actual values by actual value
    top_actual_indices = np.argsort(np.array(y_test))[-100:][::-1]
    top_100_actual_preds = all_preds[top_actual_indices]
    top_100_actual_actuals = np.array(y_test)[top_actual_indices]
    top_100_actual_names = names[top_actual_indices]

    # Calculate Precision @ top 100, Recall @ top 100, F1 @ top 100
    relevant_set = set(top_actual_indices)
    selected_set = set(top_pred_indices)

    correct_predictions = len(relevant_set.intersection(selected_set))

    # Precision, Recall, F1 calculation based on the intersection
    precision_at_100 = correct_predictions / 100  # Precision: Correct predictions in top 100 / total predictions in top 100
    recall_at_100 = correct_predictions / len(relevant_set)  # Recall: Correct predictions in top 100 / total relevant items (top 100 actuals)
    f1_at_100 = 2 * (precision_at_100 * recall_at_100) / (precision_at_100 + recall_at_100) if (precision_at_100 + recall_at_100) > 0 else 0

    # Calculate Mean Reciprocal Rank (MRR) and Average Precision at 100 (AP@100)
    mrr_at_100 = 0
    ap_at_100 = 0
    relevant_count = 0
    cumulative_precision = 0

    for idx, pred_idx in enumerate(top_pred_indices):
        if pred_idx in relevant_set:
            # For MRR, add the reciprocal rank (1/(rank+1))
            mrr_at_100 += 1 / (idx + 1)
            relevant_count += 1
            # For AP@100, add precision at this rank
            cumulative_precision += relevant_count / (idx + 1)

    # If there are relevant items, compute AP@100
    ap_at_100 = cumulative_precision / relevant_count if relevant_count > 0 else 0
    # Calculate MRR as the average reciprocal rank
    mrr_at_100 /= relevant_count if relevant_count > 0 else 1  # To avoid division by 0

    if print_results:
        print("\nMetrics at Top 100:")
        print("-" * 50)
        print(f"Precision @ Top 100: {precision_at_100:.4f}")
        print(f"Recall @ Top 100: {recall_at_100:.4f}")
        print(f"F1 @ Top 100: {f1_at_100:.4f}")
        print(f"Mean Reciprocal Rank (MRR) at 100: {mrr_at_100:.4f}")
        print(f"Average Precision at 100 (AP@100): {ap_at_100:.4f}")

    return precision_at_100, recall_at_100, f1_at_100, mrr_at_100, ap_at_100


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import re
import random
import numpy as np
from scipy.special import softmax
import pandas as pd

from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import joblib

import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig, BertConfig, AutoConfig, AutoModelForSequenceClassification

!pip install datasets
!pip install optuna
from datasets import Dataset
from sklearn.metrics import mean_squared_error
import os

import optuna

os.environ["WANDB_DISABLED"] = "true"

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==

In [5]:
usecols = ['id', 'regulationMark', 'hp', 'name', 'types', 'subtypes', 'evolvesFrom', 'evolvesTo', 'weaknesses', 'convertedRetreatCost', 'resistances', 'cleaned_attacks', 'cleaned_abilities', 'cleaned_rules', 'tournamentYear']

# Path to your datasets
pokemon_2021 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2021_pokemon_power_level.csv')
pokemon_2022 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2022_pokemon_power_level.csv')
pokemon_2023 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2023_pokemon_power_level.csv')
trainer_2021 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2021_trainer_power_level.csv')
trainer_2022 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2022_trainer_power_level.csv')
trainer_2023 = pd.read_csv('/content/drive/MyDrive/266/project/standard_2023_trainer_power_level.csv')

def print_top_10_predictions(predicted_values, actual_values, test_df):
    names = test_df['name'].values

    top_10_predicted_indices = np.argsort(predicted_values)[-10:][::-1]

    top_10_actual_indices = np.argsort(actual_values)[-10:][::-1]

    print("Top 10 Highest Predicted Values:")
    print("-" * 50)
    for i in top_10_predicted_indices:
        print(f"Name: {names[i]}, Predicted: {predicted_values[i]:.4f}, Actual: {actual_values[i]:.4f}")

    print("\nTop 10 Highest Actual Values:")
    print("-" * 50)
    for i in top_10_actual_indices:
        print(f"Name: {names[i]}, Actual: {actual_values[i]:.4f}, Predicted: {predicted_values[i]:.4f}")



In [ ]:

pokemon_2023['text'] = pokemon_2023[usecols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_df = pokemon_2023
X_test = pokemon_2023['text'].tolist()

y_test = pokemon_2023['power_level'].values

In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/pokemonModel2",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.2500
Recall @ Top 100: 0.2500
F1 @ Top 100: 0.2500
Mean Reciprocal Rank (MRR) at 100: 0.1224
Average Precision at 100 (AP@100): 0.5537


In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/pokemonRoberta",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.2700
Recall @ Top 100: 0.2700
F1 @ Top 100: 0.2700
Mean Reciprocal Rank (MRR) at 100: 0.1188
Average Precision at 100 (AP@100): 0.5741


In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/pokemonRobertaDense",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.1100
Recall @ Top 100: 0.1100
F1 @ Top 100: 0.1100
Mean Reciprocal Rank (MRR) at 100: 0.0591
Average Precision at 100 (AP@100): 0.1577


In [6]:
trainer_2023['text'] = trainer_2023[usecols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_df = trainer_2023
# X_test should be a list of strings
X_test = trainer_2023['text'].tolist()

# y_test should be the target values (can be list or array)
y_test = trainer_2023['power_level'].values

In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/trainerModel1",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.4900
Recall @ Top 100: 0.4900
F1 @ Top 100: 0.4900
Mean Reciprocal Rank (MRR) at 100: 0.0700
Average Precision at 100 (AP@100): 0.5795


In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/trainerRoberta2",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.4700
Recall @ Top 100: 0.4700
F1 @ Top 100: 0.4700
Mean Reciprocal Rank (MRR) at 100: 0.0676
Average Precision at 100 (AP@100): 0.5318


In [ ]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/trainerRobertaDense",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.5300
Recall @ Top 100: 0.5300
F1 @ Top 100: 0.5300
Mean Reciprocal Rank (MRR) at 100: 0.0451
Average Precision at 100 (AP@100): 0.4771


In [7]:
precision, recall, f1, mrr, ap = run_inference_and_print_top100(
    model_path="/content/drive/MyDrive/266/project/trainerRobertaTuned",
    X_test=X_test,
    y_test=y_test,
    test_df=test_df
)


Metrics at Top 100:
--------------------------------------------------
Precision @ Top 100: 0.5100
Recall @ Top 100: 0.5100
F1 @ Top 100: 0.5100
Mean Reciprocal Rank (MRR) at 100: 0.0674
Average Precision at 100 (AP@100): 0.6057
